In [ ]:
install.packages('readr')
install.packages("tidyverse")
install.packages("skimr")
install.packages("janitor")
install.packages('kimisc')
install.packages('dplyr')
install.packages('stringr')
install.packages('lubridate')
install.packages('hms')
install.packages('ragtop')
install.packages("writexl")
install.packages("geosphere")

In [49]:
library(readr)
library(kimisc)
library(dplyr)
library(stringr)
library(tidyverse)
library(skimr)
library(janitor)
library(lubridate)
library(hms)
library(ragtop)
library(writexl)
library(geosphere)

Warning message:
"package 'geosphere' was built under R version 4.2.1"


The business task is to determine how casual riders and Divvy members use Divvy company bikes differently, in order to determine how to convert casual riders into members. In order to do this, we need to find out how casual riders use the bikes, and how members use the bikes. Key stakeholders include:
Divvy: A bike-share program that features more than 5,800 bicycles and 600 docking stations. Note: Divvy was purchased by Lyft in 2019.
Lily Moreno: The director of marketing and your manager. Moreno is responsible for the development of campaigns and initiatives to promote the bike-share program. These may include email, social media, and other channels.
Divvy marketing analytics team: A team of data analysts who are responsible for collecting, analyzing, and reporting data that helps guide Divvy marketing strategy.
Divvy executive team: The notoriously detail-oriented executive team will decide whether to approve the recommended marketing program


The data is located here: https://divvy-tripdata.s3.amazonaws.com/index.html

Looking at the data, the data is organized in various chunks of time, but there is a continuous stream of data from 2013 through the end of 2021. When looking at data and questioning it's quality, we use ROCCC: The data is reliable because it is first party data collected by the company requesting the analysis. It does not have selection bias, as our question is only looking at current customers, and the dataset includes only customers, and all customer trips during the selected period. The data we will choose is over the course of 2021, as that is a large enough data set post-pandemic.

In [50]:
# First, we import the data and replace any blank cells with NAs. 
raw_data <- rbind(read_csv("Divvy Trip Data 2021/202101-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202102-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202103-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202104-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202105-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202106-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202107-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202108-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202109-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202110-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202111-divvy-tripdata.csv", na=c(""," ","NA")),
                  read_csv("Divvy Trip Data 2021/202112-divvy-tripdata.csv", na=c(""," ","NA")))

Rows: 96834 Columns: 13
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (7): ride_id, rideable_type, start_station_name, start_station_id, end_...
dbl  (4): start_lat, start_lng, end_lat, end_lng
dttm (2): started_at, ended_at

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 49622 Columns: 13
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (7): ride_id, rideable_type, start_station_name, start_station_id, end_...
dbl  (4): start_lat, start_lng, end_lat, end_lng
dttm (2): started_at, ended_at

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 228496 Columns: 13
── Column specification ─

In [61]:
glimpse(raw_data)

Rows: 5,595,063
Columns: 15
$ ride_id            <chr> "E19E6F1B8D4C42ED", "DC88F20C2C55F27F", "EC45C94683…
$ rideable_type      <chr> "electric_bike", "electric_bike", "electric_bike", …
$ started_at         <dttm> 2021-01-23 16:14:19, 2021-01-27 18:43:08, 2021-01-…
$ ended_at           <dttm> 2021-01-23 16:24:44, 2021-01-27 18:47:12, 2021-01-…
$ start_station_name <chr> "California Ave & Cortez St", "California Ave & Cor…
$ start_station_id   <chr> "17660", "17660", "17660", "17660", "17660", "17660…
$ end_station_name   <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, "Wood St & Augu…
$ end_station_id     <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, "657", "13258",…
$ start_lat          <dbl> 41.90034, 41.90033, 41.90031, 41.90040, 41.90033, 4…
$ start_lng          <dbl> -87.69674, -87.69671, -87.69664, -87.69666, -87.696…
$ end_lat            <dbl> 41.89000, 41.90000, 41.90000, 41.92000, 41.90000, 4…
$ end_lng            <dbl> -87.72000, -87.69000, -87.70000, -87.69000, -87.700…
$ member_cas

In [69]:
# Next, we calc. the length of each ride in seconds
raw_data$ride_length <- as.integer(difftime(raw_data$ended_at, raw_data$started_at, units="secs"))
# And we calc.the distance of each ride in meters
raw_data$ride_distance <- distHaversine(raw_data[,9:10], raw_data[,11:12]) 
# And store the time, date, day, month and year of the ride.
raw_data$time <-  format(as.POSIXct(raw_data$started_at), format = "%H:%M:%S")
raw_data$date <- as.Date(raw_data$started_at) 
raw_data$month <- format(as.Date(raw_data$date), "%m")
raw_data$day <- format(as.Date(raw_data$date), "%d")
raw_data$year <- format(as.Date(raw_data$date), "%Y")
# Then, we create a column designating what day of the week the ride took place on. 
raw_data$day_of_week <- weekdays(as.Date(raw_data$date))
# Next, we remove any rows that are entirely blank.
raw_data <- raw_data[rowSums(is.na(raw_data)) != ncol(raw_data),]

In [70]:
str(raw_data)

tibble [5,595,063 × 21] (S3: tbl_df/tbl/data.frame)
 $ ride_id           : chr [1:5595063] "E19E6F1B8D4C42ED" "DC88F20C2C55F27F" "EC45C94683FE3F27" "4FA453A75AE377DB" ...
 $ rideable_type     : chr [1:5595063] "electric_bike" "electric_bike" "electric_bike" "electric_bike" ...
 $ started_at        : POSIXct[1:5595063], format: "2021-01-23 16:14:19" "2021-01-27 18:43:08" ...
 $ ended_at          : POSIXct[1:5595063], format: "2021-01-23 16:24:44" "2021-01-27 18:47:12" ...
 $ start_station_name: chr [1:5595063] "California Ave & Cortez St" "California Ave & Cortez St" "California Ave & Cortez St" "California Ave & Cortez St" ...
 $ start_station_id  : chr [1:5595063] "17660" "17660" "17660" "17660" ...
 $ end_station_name  : chr [1:5595063] NA NA NA NA ...
 $ end_station_id    : chr [1:5595063] NA NA NA NA ...
 $ start_lat         : num [1:5595063] 41.9 41.9 41.9 41.9 41.9 ...
 $ start_lng         : num [1:5595063] -87.7 -87.7 -87.7 -87.7 -87.7 ...
 $ end_lat           : num [1:5595063] 

In [86]:
sprintf("The longest member ride distance is: %d meters", 
        as.integer(max(as.integer(!is.na(raw_data$ride_distance)))))

[1] "The longest member ride distance is: 1 meters"

In [71]:
# Let's split the data into member and casual rides, and look at the max, min, mean and median of each.
member_data <- subset(raw_data, raw_data$member_casual == "member") 
casual_data <- subset(raw_data, raw_data$member_casual == "casual") 

In [72]:
sprintf("The mean of member ride length is: %d seconds", 
        as.integer(mean(as.integer(member_data$ride_length))))
sprintf("The mean of casual ride length is: %d seconds", 
        as.integer(mean(as.integer(casual_data$ride_length))))

sprintf("The median of member ride length is: %d seconds", 
        as.integer(median(as.integer(member_data$ride_length))))
sprintf("The median of casual ride length is: %d seconds", 
        as.integer(median(as.integer(casual_data$ride_length))))

sprintf("The longest member ride length is: %d seconds", 
        as.integer(max(as.integer(member_data$ride_length))))
sprintf("The longest casual ride length is: %d seconds", 
        as.integer(max(as.integer(casual_data$ride_length))))

sprintf("The longest member ride length is: %d seconds", 
        as.integer(min(as.integer(member_data$ride_length))))
sprintf("The longest casual ride length is: %d seconds", 
        as.integer(min(as.integer(casual_data$ride_length))))

[1] "The mean of member ride length is: 817 seconds"

[1] "The mean of casual ride length is: 1920 seconds"

[1] "The median of member ride length is: 576 seconds"

[1] "The median of casual ride length is: 958 seconds"

[1] "The longest member ride length is: 93596 seconds"

[1] "The longest casual ride length is: 3356649 seconds"

[1] "The longest member ride length is: -3245 seconds"

[1] "The longest casual ride length is: -3482 seconds"

In [73]:
# We can see that some rides have a negative ride length, and the longest ride has a duration of 38.85 days.
# So, let's remove any rides that have a negative ride length, or are over 24 hours long.
# (This is still a very generous upper max for ride length, but it ensures that riders that are docking their 
# bikes and come back after work are always counted.)
raw_data <- raw_data[raw_data$ride_length >= 0 | raw_data$ride_length < 60*60*24,]

In [74]:
str(raw_data)

tibble [5,595,063 × 21] (S3: tbl_df/tbl/data.frame)
 $ ride_id           : chr [1:5595063] "E19E6F1B8D4C42ED" "DC88F20C2C55F27F" "EC45C94683FE3F27" "4FA453A75AE377DB" ...
 $ rideable_type     : chr [1:5595063] "electric_bike" "electric_bike" "electric_bike" "electric_bike" ...
 $ started_at        : POSIXct[1:5595063], format: "2021-01-23 16:14:19" "2021-01-27 18:43:08" ...
 $ ended_at          : POSIXct[1:5595063], format: "2021-01-23 16:24:44" "2021-01-27 18:47:12" ...
 $ start_station_name: chr [1:5595063] "California Ave & Cortez St" "California Ave & Cortez St" "California Ave & Cortez St" "California Ave & Cortez St" ...
 $ start_station_id  : chr [1:5595063] "17660" "17660" "17660" "17660" ...
 $ end_station_name  : chr [1:5595063] NA NA NA NA ...
 $ end_station_id    : chr [1:5595063] NA NA NA NA ...
 $ start_lat         : num [1:5595063] 41.9 41.9 41.9 41.9 41.9 ...
 $ start_lng         : num [1:5595063] -87.7 -87.7 -87.7 -87.7 -87.7 ...
 $ end_lat           : num [1:5595063] 

In [75]:
# Now, let's look at the rest of the variables. It's important to validate our data before we analyze it,
# because if it's not in the correct format, or it's incorrect it can ruin the analysis. 

# The ride_id is the primary key, and so each value should be unique, so we should expect there to be 
# as many different ride ID's as there are rides in the dataset:
sprintf('the number of ride IDs is: %d', length(unique(raw_data$ride_id)))
sprintf("The number of rides in the dataset is: %d", nrow(raw_data))

# Divvy offers electric and regular bikes, 
# and looking at Divvy's website there is also the option to dock a bike to reserve it for later use.
print(unique(raw_data$rideable_type))

# All start and stop times for each ride should be during the year 2021:
sprintf("The most recent ride in the dataset took place at: %s", 
        max(raw_data$ended_at))
sprintf("The least recent ride in the dataset took place at: %s", 
        min(raw_data$started_at))

# We see that there is at least one ride that finished in 2022. So, lets filter out any rides not in 2021: 
raw_data <- raw_data[raw_data$year == "2021",]

# Next, we know that customers can start or stop their ride from any location that has a bike, 
# whether or not that location is a station. However, let's see if there
# are any incosistancies in the station names or ID's:

[1] "the number of ride IDs is: 5595063"

[1] "The number of rides in the dataset is: 5595063"

[1] "electric_bike" "classic_bike"  "docked_bike"  


[1] "The most recent ride in the dataset took place at: 2022-01-03 17:32:18"

[1] "The least recent ride in the dataset took place at: 2021-01-01 00:02:05"

In [76]:
raw_data[is.na(raw_data$start_station_name) & !is.na(raw_data$start_station_id),] 

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,⋯,end_lng,member_casual,ride_length,ride_distance,time,date,month,day,year,day_of_week
<chr>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<int>,<dbl>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>
176105D1F8A1216B,electric_bike,2021-07-18 03:44:22,2021-07-18 04:12:23,NA,13221,NA,NA,41.90765,-87.67255,⋯,-87.74000,casual,1681,7510.64631,03:44:22,2021-07-18,07,18,2021,Sunday
DE82A15026BA3056,electric_bike,2021-09-21 18:18:59,2021-09-21 18:21:48,NA,20215,Hegewisch Metra Station,20215,41.64850,-87.54609,⋯,-87.54625,casual,169,17.89034,18:18:59,2021-09-21,09,21,2021,Tuesday
EE197EDA4CF8CFE5,electric_bike,2021-09-22 07:14:42,2021-09-22 07:22:38,NA,WL-008,NA,NA,41.86712,-87.64109,⋯,-87.64000,casual,476,134.63987,07:14:42,2021-09-22,09,22,2021,Wednesday


In [77]:
raw_data[!is.na(raw_data$start_station_name) & is.na(raw_data$start_station_id),] 

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"


ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,⋯,end_lng,member_casual,ride_length,ride_distance,time,date,month,day,year,day_of_week
<chr>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<int>,<dbl>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>


In [78]:
raw_data[is.na(raw_data$end_station_name) & !is.na(raw_data$end_station_id),] 

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"


ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,⋯,end_lng,member_casual,ride_length,ride_distance,time,date,month,day,year,day_of_week
<chr>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<int>,<dbl>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>


In [79]:
raw_data[!is.na(raw_data$end_station_name) & is.na(raw_data$end_station_id),] 

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"


ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,⋯,end_lng,member_casual,ride_length,ride_distance,time,date,month,day,year,day_of_week
<chr>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<int>,<dbl>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>


we see three examples with a start station id, and no start station name,so lets just go ahead and remove them:

In [80]:
raw_data <- raw_data[!(is.na(raw_data$start_station_name) & !is.na(raw_data$start_station_id)),]

In [81]:
write.csv(raw_data,"Divvy_Bike_Rides_2021_Cleaned.csv")